In [1]:
#!pip install scikit-learn --force
#!pip install catboost

In [2]:
import sys
sys.path.append('../..')
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd
from datetime import datetime
import os
import importlib

pd.set_option('display.max.columns', 300)

from core.calculator.storage import ModelDB
from core.calculator.core import ForecastConfig
from core.calculator.core import TrainingManager
from core.calculator.core import ForecastConfig
from core.calculator.core import ForecastEngine

from core.calculator.deposits import DepositsCalculationType
from core.calculator.deposits import DepositIterativeCalculator

from core.definitions import *
from core.project_update import load_portfolio

from core.models import DepositModels
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
# если хотим обучить модели

from core.models.utils import run_spark_session
#spark = run_spark_session('check_calc')

spark = None #если без обучения

In [4]:
# Депозиты
# portfolio_res = engine.calc_results['Deposits']['portfolio'] 
# agg_res = engine.calc_results['Deposits']['agg_data']

In [5]:
# engine.calc_results['Deposits']['portfolio'].head()

In [6]:
# engine.calc_results['Deposits']['agg_data'].head()

In [7]:
# engine.calc_results['SavingAccounts'] # накопительные счета

## Калькулятор маржи

При расчете идет усреднение за большой промежуток времени (1 год и 3 года). В первом приближении предполагаем что значения по ставки сильно не меняются и основное для нас - правильный прогноз, от него будем считать все необходимые значения с умножением на трансфертную ставку

In [8]:
ca_fact = pd.read_csv('Current_accounts.csv')

In [9]:
ca_fact.head()

,report_dt,balance
0,2023-08-31 00:00:00,7.334889e+11
1,2023-08-30 00:00:00,7.230378e+11
2,2023-08-29 00:00:00,7.177987e+11
3,2023-08-28 00:00:00,7.122633e+11
4,2023-08-27 00:00:00,6.979975e+11


In [10]:
ca_agg = pd.read_csv('ca_agg_balance_v2.csv')

In [11]:
ca_agg = ca_agg[(ca_agg['report_dt']<'2023-08-31')&(ca_agg['report_dt']>='2023-01-01')]

In [12]:
ca_agg.head(10)

,report_dt,balance_eur_[general],balance_oth_[general],balance_rur_[general],balance_usd_[general],balance_rur_[vip],balance_rur_[priv],balance_rur_[mass],rate_ruonia,vtb_ftp_rate_[90d],vtb_ftp_rate_[730d]
1,2023-07-31 00:00:00,6.054338e+08,7.213652e+09,6.794145e+11,1.215198e+09,3.729472e+10,1.410197e+11,5.011001e+11,7.830476,8.407143,9.278571
2,2023-06-30 00:00:00,6.377248e+08,5.303443e+09,6.426842e+11,1.256810e+09,3.331914e+10,1.367460e+11,4.726191e+11,7.292857,8.100000,9.150000
3,2023-05-31 00:00:00,6.555641e+08,5.233935e+09,6.104075e+11,1.281959e+09,2.917961e+10,1.218032e+11,4.594247e+11,7.211500,8.050000,9.150000
4,2023-04-30 00:00:00,6.782228e+08,4.935703e+09,5.880988e+11,1.317959e+09,2.824127e+10,1.185661e+11,4.412914e+11,7.149500,8.050000,9.150000
5,2023-03-31 00:00:00,7.042712e+08,5.305270e+09,5.667109e+11,1.374661e+09,2.847208e+10,1.145350e+11,4.237039e+11,7.339091,8.050000,9.150000
6,2023-02-28 00:00:00,7.247016e+08,3.493589e+09,5.415421e+11,1.390496e+09,2.555346e+10,1.095706e+11,4.064180e+11,7.135000,8.005556,9.150000
7,2023-01-31 00:00:00,7.434590e+08,3.134237e+09,5.529541e+11,1.390937e+09,2.626417e+10,1.127686e+11,4.139213e+11,7.000000,8.000000,9.150000


In [13]:
import pip

In [14]:
#pip.main(['install', 'openpyxl'])

In [15]:
#ca_agg.to_excel('ca_agg_balance_v2.xlsx')

In [16]:
# вбиваем с конца

rate = pd.Series([8.5, 8.23, 8.18, 8.18, 8.12, 8.12, 8.12])
SSV = 0.48
FOR =  pd.Series([4.5, 4.5, 4, 4, 4, 3, 3])

#weighted_rate =  pd.Series([0.49, 0.48, 0.5, 0.49, 0.51, 0.48, 0.48])

weighted_rate = 0.49

In [17]:
ftp = rate * (1- FOR / 100) - SSV - weighted_rate

In [18]:
ftp

0    7.14750
1    6.88965
2    6.88280
3    6.88280
4    6.82520
5    6.90640
6    6.90640
dtype: float64

In [19]:
# для помесячной истории


ftp_month = ((1+ftp/100)**(1/12))

In [20]:
ftp_day = ((ftp/100)*(1/365))

In [21]:
ftp_day

0    0.000196
1    0.000189
2    0.000189
3    0.000189
4    0.000187
5    0.000189
6    0.000189
dtype: float64

In [22]:
ca_agg['perc_general'] = ca_agg['balance_rur_[general]'] * (ftp_month-1)
ca_agg['perc_vip'] = ca_agg['balance_rur_[vip]'] * (ftp_month-1)
ca_agg['perc_priv'] = ca_agg['balance_rur_[priv]'] * (ftp_month-1)
ca_agg['perc_mass'] = ca_agg['balance_rur_[mass]'] * (ftp_month-1)

In [23]:
ca_agg['report_dt'].iloc[0][8:10]

'31'

In [24]:
ftp_day

0    0.000196
1    0.000189
2    0.000189
3    0.000189
4    0.000187
5    0.000189
6    0.000189
dtype: float64

In [25]:
ca_agg['report_dt'].apply(lambda x: int(x[8:10]))

1    31
2    30
3    31
4    30
5    31
6    28
7    31
Name: report_dt, dtype: int64

In [26]:
(((ftp_day)*(ca_agg['report_dt'].apply(lambda x: int(x[8:10]))).values)).values

array([0.00607048, 0.00566273, 0.00584567, 0.0056571 , 0.00579675,
       0.00529806, 0.00586571])

In [27]:
ftp_day-1

0   -0.999804
1   -0.999811
2   -0.999811
3   -0.999811
4   -0.999813
5   -0.999811
6   -0.999811
dtype: float64

In [28]:
ca_agg['report_dt'].apply(lambda x: int(x[8:10])).values

array([31, 30, 31, 30, 31, 28, 31])

In [29]:
ca_agg['perc_general'] = ca_agg['balance_rur_[general]'] * (((ftp_day)*(ca_agg['report_dt'].apply(lambda x: int(x[8:10]))).values)).values
ca_agg['perc_vip'] = ca_agg['balance_rur_[vip]'] * (((ftp_day)*(ca_agg['report_dt'].apply(lambda x: int(x[8:10]))).values)).values
ca_agg['perc_priv'] = ca_agg['balance_rur_[priv]'] * (((ftp_day)*(ca_agg['report_dt'].apply(lambda x: int(x[8:10]))).values)).values
ca_agg['perc_mass'] = ca_agg['balance_rur_[mass]'] * (((ftp_day)*(ca_agg['report_dt'].apply(lambda x: int(x[8:10]))).values)).values

In [30]:
ftp_month - 1

0    0.005770
1    0.005568
2    0.005562
3    0.005562
4    0.005517
5    0.005581
6    0.005581
dtype: float64

In [31]:
ca_agg.sort_values(by='report_dt')[['report_dt', 'perc_general', 'perc_vip', 'perc_priv', 'perc_mass']].T

,7,6,5,4,3,2,1
report_dt,2023-01-31 00:00:00,2023-02-28 00:00:00,2023-03-31 00:00:00,2023-04-30 00:00:00,2023-05-31 00:00:00,2023-06-30 00:00:00,2023-07-31 00:00:00
perc_general,3243468254.741209,2869122448.003284,3285078800.273827,3326931065.278562,3568238301.551342,3639344739.293152,4124371636.512539
perc_vip,154057982.104066,135383779.562126,165045399.362485,159763582.438875,170574254.516088,188677135.90854,226396826.315981
perc_priv,661467873.870684,580511557.148642,663930080.650274,670739956.078713,712020953.858372,774355194.917042,856057119.631611
perc_mass,2427942398.766458,2153227111.292517,2456103320.261068,2496427526.760973,2685643093.176882,2676312408.467571,3041917690.564947


In [36]:
#ca_agg.sort_values(by='report_dt')[['report_dt', 'perc_general', 'perc_vip', 'perc_priv', 'perc_mass']].T.to_excel('ca_agg_res_v4.xlsx')

In [33]:
# ca_agg.to_excel('ca_agg_res.xlsx')

Расчет идет от стабильной или волатильной части?

In [34]:
#ca_agg['perc_margin'] = ca_agg['perc'] / ca_agg['balance_rur_[general]'] * 365 / 30.3 * 100

In [35]:
#ca_agg